# KNN Cluster

# Import data and libraries

In [1]:
import pandas as pd, numpy as np, warnings
from scipy.io import arff

In [2]:
data = arff.loadarff('segment.arff')
df = pd.DataFrame(data[0])
df.head()

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean,class
0,218.0,178.0,9.0,0.111111,0.0,0.833333,0.547722,1.111110,0.544331,59.629600,52.4444,75.22220,51.222200,-21.55560,46.7778,-25.22220,75.22220,0.318996,-2.04055,b'path'
1,113.0,130.0,9.0,0.000000,0.0,0.277778,0.250924,0.333333,0.365148,0.888889,0.0000,2.55556,0.111111,-2.66667,5.0000,-2.33333,2.55556,1.000000,-2.12325,b'foliage'
2,202.0,41.0,9.0,0.000000,0.0,0.944448,0.772202,1.111110,1.025600,123.037000,111.8890,139.77800,117.444000,-33.44440,50.2222,-16.77780,139.77800,0.199347,-2.29992,b'sky'
3,32.0,173.0,9.0,0.000000,0.0,1.722220,1.781590,9.000000,6.749490,43.592600,39.5556,52.88890,38.333300,-12.11110,27.8889,-15.77780,52.88890,0.266914,-1.99886,b'path'
4,61.0,197.0,9.0,0.000000,0.0,1.444440,1.515350,2.611110,1.925460,49.592600,44.2222,61.55560,43.000000,-16.11110,35.8889,-19.77780,61.55560,0.302925,-2.02227,b'path'


# Z-score normalize

In [3]:
X = df.iloc[:,:-1]
X = (X - X.mean()) / X.std()
X.replace(np.NaN, 0, inplace=True)
X.head()

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean
0,1.275912,0.949531,0.0,2.410147,-0.19451,-0.392976,-0.115095,-0.363873,-0.130916,0.591412,0.560071,0.712983,0.469592,-0.765286,1.296198,-1.428735,0.700920,-0.472591,-0.438515
1,-0.163301,0.114514,0.0,-0.356970,-0.19451,-0.598820,-0.121713,-0.579319,-0.133963,-0.947253,-0.936767,-0.956461,-0.935919,0.865381,-0.838392,0.552706,-0.992082,2.510221,-0.492031
2,1.056604,-1.433747,0.0,-0.356970,-0.19451,-0.351806,-0.110089,-0.363873,-0.122733,2.252318,2.256706,2.196088,2.290634,-1.791637,1.472186,-0.697721,2.204955,-0.996656,-0.606356
3,-1.273551,0.862550,0.0,-0.356970,-0.19451,-0.063626,-0.087582,1.821364,-0.025407,0.171336,0.192206,0.199897,0.115158,0.050050,0.331091,-0.611153,0.180594,-0.700711,-0.411537
4,-0.876054,1.280058,0.0,-0.356970,-0.19451,-0.166549,-0.093518,0.051629,-0.107432,0.328501,0.325398,0.399006,0.243488,-0.295267,0.739842,-0.957424,0.382513,-0.542982,-0.426686


In [4]:
y = df.iloc[:,-1]
y.value_counts()

b'grass'        330
b'sky'          330
b'foliage'      330
b'window'       330
b'brickface'    330
b'path'         330
b'cement'       330
Name: class, dtype: int64

# Centroid Indexes

In [5]:
centroid_points_or = [775, 1020, 200, 127, 329, 1626, 1515, 651, 658, 328, 1160, 108, 422, 88, 
                   105, 261, 212, 1941, 1724, 704, 1469, 635, 867, 1187, 445, 222, 1283, 
                   1288, 1766, 1168, 566, 1812, 214, 53, 423, 50, 705, 1284, 1356, 996, 
                   1084, 1956, 254, 711, 1997, 1378, 827, 1875, 424, 1790, 633, 208, 1670, 
                   1517, 1902, 1476, 1716, 1709, 264, 1, 371, 758, 332, 542, 672, 483, 65, 
                   92, 400, 1079, 1281, 145, 1410, 664, 155, 166, 1900, 1134, 1462, 954, 1818,
                   1679, 832, 1627, 1760, 1330, 913, 234, 1635, 1078, 640, 833, 392, 1425, 610,
                   1353, 1772, 908, 1964, 1260, 784, 520, 1363, 544, 426, 1146, 987, 612, 1685,
                   1121, 1740, 287, 1383, 1923, 1665, 19, 1239, 251, 309, 245, 384, 1306, 786, 
                   1814, 7, 1203, 1068, 1493, 859, 233, 1846, 1119, 469, 1869, 609, 385, 
                   1182, 1949, 1622, 719, 643, 1692, 1389, 120, 1034, 805, 266, 339,
 826, 530, 1173, 802, 1495, 504, 1241, 427, 1555, 1597, 692, 178, 774, 1623, 1641, 661,
 1242, 1757, 553, 1377, 1419, 306, 1838, 211, 356, 541, 1455, 741, 583, 1464, 209, 1615,
 475, 1903, 555, 1046, 379, 1938, 417, 1747, 342, 1148, 1697, 1785, 298, 1485, 945, 1097, 
 207, 857, 1758, 1390, 172, 587, 455, 1690, 1277, 345, 1166, 1367, 1858, 1427, 1434, 953, 
 1992, 1140, 137, 64, 1448, 991, 1312, 1628, 167, 1042, 1887, 1825, 249, 240, 524, 1098, 
 311, 337, 220, 1913, 727, 1659, 1321, 130, 1904, 561, 1270, 1250, 613, 152, 1440, 473, 
 1834, 1387, 1656, 1028, 1106, 829, 1591, 1699, 1674, 947, 77, 468, 997, 611, 1776, 123, 
 979, 1471, 1300, 1007, 1443, 164, 1881, 1935, 280, 442, 1588, 1033, 79, 1686, 854, 257, 
 1460, 1380, 495, 1701, 1611, 804, 1609, 975, 1181, 582, 816, 1770, 663, 737, 1810, 523, 
 1243, 944, 1959, 78, 675, 135, 1381, 1472]

In [6]:
centroid_points = [x-1 for x in centroid_points_or]

# Create centroid list

In [7]:
def k_means(k):
    
    '''Returns 25 random index sets
    of k centroids each'''
    
    centroid_list = []
    for i in range(60):
        if i > 24:
            break
        else:
            start = i*k
            end = ((i+1)*k)
            centroid_list.append(centroid_points[start:end])
    return centroid_list


# Centroid Creation

In [8]:
def assign_centroid(n,k):
   
    '''
    Returns list of intial k centroids
    based on a list of random indices (subset n)
    
    n determines which of the 25 subsets of random indices is returned
    k determines how many k centroids are generated
    '''
    end = k  
    centroid_list = k_means(k)
    
    a = 0
    b = 0 
    c = 0
    d = 0
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    j = 0
    k = 0
    l = 0
    
    try:
        a = X.iloc[centroid_list[n],:].iloc[0]
        b = X.iloc[centroid_list[n],:].iloc[1]
        c = X.iloc[centroid_list[n],:].iloc[2]
        d = X.iloc[centroid_list[n],:].iloc[3]
        e = X.iloc[centroid_list[n],:].iloc[4]
        f = X.iloc[centroid_list[n],:].iloc[5]
        g = X.iloc[centroid_list[n],:].iloc[6]
        h = X.iloc[centroid_list[n],:].iloc[7]
        i = X.iloc[centroid_list[n],:].iloc[8]
        j = X.iloc[centroid_list[n],:].iloc[9]
        k = X.iloc[centroid_list[n],:].iloc[10]
        l = X.iloc[centroid_list[n],:].iloc[11]
    
        centroids = [a,b,c,d,e,f,g,h,i,j,k,l]
        
    
    except:
        
        
        centroids = [a,b,c,d,e,f,g,h,i,j,k,l]
    
    return centroids[:end]
    

# Class Membership

In [9]:
def class_membership(X, centroids, k):
   
    end = k
    centroid_class = []
    
    a_dist = 1000
    b_dist = 1000
    c_dist = 1000
    d_dist = 1000
    e_dist = 1000
    f_dist = 1000
    g_dist = 1000
    h_dist = 1000
    i_dist = 1000
    j_dist = 1000
    k_dist = 1000
    l_dist = 1000
    
    for i in range(len(X)):
       
        try:
            a_dist = np.sqrt(np.sum(np.square(centroids[0] - X.iloc[i,:])))
            b_dist = np.sqrt(np.sum(np.square(centroids[1] - X.iloc[i,:])))
            c_dist = np.sqrt(np.sum(np.square(centroids[2] - X.iloc[i,:])))
            d_dist = np.sqrt(np.sum(np.square(centroids[3] - X.iloc[i,:])))
            e_dist = np.sqrt(np.sum(np.square(centroids[4] - X.iloc[i,:])))
            f_dist = np.sqrt(np.sum(np.square(centroids[5] - X.iloc[i,:])))
            g_dist = np.sqrt(np.sum(np.square(centroids[6] - X.iloc[i,:])))
            h_dist = np.sqrt(np.sum(np.square(centroids[7] - X.iloc[i,:])))
            i_dist = np.sqrt(np.sum(np.square(centroids[8] - X.iloc[i,:])))
            j_dist = np.sqrt(np.sum(np.square(centroids[9] - X.iloc[i,:])))
            k_dist = np.sqrt(np.sum(np.square(centroids[10] - X.iloc[i,:])))
            l_dist = np.sqrt(np.sum(np.square(centroids[11] - X.iloc[i,:])))
        
            res = np.min([a_dist, b_dist, c_dist, d_dist, e_dist,
                             f_dist, g_dist, h_dist, i_dist, j_dist, 
                             k_dist, l_dist])
            
            if res == a_dist:
                cen = 'a'
            if res == b_dist:
                cen = 'b'
            if res == c_dist:
                cen = 'c'    
            if res == d_dist:
                cen = 'd'
            if res == e_dist:
                cen = 'e'
            if res == f_dist:
                cen = 'f'
            if res == g_dist:
                cen = 'g'
            if res == h_dist:
                cen = 'h'
            if res == i_dist:
                cen = 'i'
            if res == j_dist:
                cen = 'j'
            if res == k_dist:
                cen = 'k'
            if res == l_dist:
                cen = 'l'
                
            centroid_class.append(cen)
            
        except:
            
            distances = [a_dist, b_dist, c_dist, d_dist, e_dist,
                             f_dist, g_dist, h_dist, i_dist, j_dist, 
                             k_dist, l_dist]
            
            dist_sub = distances[:k]

            res = np.min(dist_sub)

            if res == a_dist:
                cen = 'a'
            if res == b_dist:
                cen = 'b'
            if res == c_dist:
                cen = 'c'   
            if res == d_dist:
                cen = 'd'
            if res == e_dist:
                cen = 'e'
            if res == f_dist:
                cen = 'f'
            if res == g_dist:
                cen = 'g'
            if res == h_dist:
                cen = 'h'
            if res == i_dist:
                cen = 'i'
            if res == j_dist:
                cen = 'j'
            if res == k_dist:
                cen = 'k'
            if res == l_dist:
                cen = 'l'

            centroid_class.append(cen)
            
    return pd.Series(centroid_class)

# Define new centroids based on class member average

In [10]:
def center_centroid(X, run, k):
    
    end = k
    new_a = 0
    new_b = 0
    new_c = 0
    new_d = 0
    new_e = 0
    new_f = 0
    new_g = 0
    new_h = 0
    new_i = 0
    new_j = 0
    new_k = 0
    new_l = 0
    
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
    
        index_a = list(run[run == 'a'].index)
        new_a = np.average(X.ix[index_a], axis = 0)
        
        index_b = list(run[run == 'b'].index)
        new_b = np.average(X.ix[index_b], axis = 0)
        
        index_c = list(run[run == 'c'].index)
        new_c = np.average(X.ix[index_c], axis = 0)
        
        index_d = list(run[run == 'd'].index)
        new_d = np.average(X.ix[index_d], axis = 0)
        
        index_e = list(run[run == 'e'].index)
        new_e = np.average(X.ix[index_e], axis = 0)
        
        index_f = list(run[run == 'f'].index)
        new_f = np.average(X.ix[index_f], axis = 0)
        
        index_g = list(run[run == 'g'].index)
        new_g = np.average(X.ix[index_g], axis = 0)
        
        index_h = list(run[run == 'h'].index)
        new_h = np.average(X.ix[index_h], axis = 0)
        
        index_i = list(run[run == 'i'].index)
        new_i = np.average(X.ix[index_i], axis = 0)
        
        index_j = list(run[run == 'j'].index)
        new_j = np.average(X.ix[index_j], axis = 0)
        
        index_k = list(run[run == 'k'].index)
        new_k = np.average(X.ix[index_k], axis = 0)
        
        index_l = list(run[run == 'l'].index)
        new_l = np.average(X.ix[index_l], axis = 0)

        new_centroids = [new_a, new_b, new_c, new_d, new_e, new_f, new_g, 
                        new_h, new_i, new_j, new_k, new_l]
        
        new_centroids_sub = new_centroids[:end]
        
        return new_centroids_sub

# K Means Cluster

In [11]:
def k_means_cluster(X,k,n): # add 0 for n argument
    
    initial_indices = assign_centroid(n,k) # replace 0 with n
    first_run = class_membership(X, initial_indices, k)
    first_recentering = center_centroid(X, first_run, k)
    
    centering = first_recentering
    
    centroid_movement = [centering]
    
    count = 0
    
    for i in range(49):
        
        run = class_membership(X, centering, k)
        
        centering = center_centroid(X, run, k)
        

        centroid_movement.append(centering)
        
        count = count + 1 
        
        if centroid_movement[i][0].tolist() == centroid_movement[i+1][0].tolist() or (count + 1) > 50:  
            break
            
    print('algorithm ran ' + str(count) + ' times')       
    return (centering, run)

# SSE_Score

In [12]:
def SSE_SCORE(X, test):
    
    new_a = 0
    new_b = 0
    new_c = 0
    new_d = 0
    new_e = 0
    new_f = 0
    new_g = 0
    new_h = 0
    new_i = 0
    new_j = 0
    new_k = 0
    new_l = 0
    
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        try:
            
            index_a = list(test[1][test[1] == 'a'].index)
            new_a = np.nansum(np.square(np.abs(X.ix[index_a] - test[0][0])))

            index_b = list(test[1][test[1] == 'b'].index)
            new_b = np.nansum(np.square(np.abs(X.ix[index_b] - test[0][1])))

            index_c = list(test[1][test[1] == 'c'].index)
            new_c = np.nansum(np.square(np.abs(X.ix[index_c] - test[0][2])))

            index_d = list(test[1][test[1] == 'd'].index)
            new_d = np.nansum(np.square(np.abs(X.ix[index_d] - test[0][3])))

            index_e = list(test[1][test[1] == 'e'].index)
            new_e = np.nansum(np.square(np.abs(X.ix[index_e] - test[0][4])))

            index_f = list(test[1][test[1] == 'f'].index)
            new_f = np.nansum(np.square(np.abs(X.ix[index_f] - test[0][5])))

            index_g = list(test[1][test[1] == 'g'].index)
            new_g = np.nansum(np.square(np.abs(X.ix[index_g] - test[0][6])))

            index_h = list(test[1][test[1] == 'h'].index)
            new_h = np.nansum(np.square(np.abs(X.ix[index_h] - test[0][7])))

            index_i = list(test[1][test[1] == 'i'].index)
            new_i = np.nansum(np.square(np.abs(X.ix[index_i] - test[0][8])))

            index_j = list(test[1][test[1] == 'j'].index)
            new_j = np.nansum(np.square(np.abs(X.ix[index_j] - test[0][9])))

            index_k = list(test[1][test[1] == 'k'].index)
            new_k = np.nansum(np.square(np.abs(X.ix[index_k] - test[0][10])))

            index_l = list(test[1][test[1] == 'l'].index)
            new_l = np.nansum(np.square(np.abs(X.ix[index_l] - test[0][11])))
            
            distances = [new_a, new_b, new_c, new_d, new_e, new_f, new_g, 
                        new_h, new_i, new_j, new_k, new_l]
            SSE_total = np.nansum(distances)
        
        except:
            
            distances = [new_a, new_b, new_c, new_d, new_e, new_f, new_g, 
                        new_h, new_i, new_j, new_k, new_l]
            SSE_total = np.nansum(distances)
        

        return SSE_total

# K means score

In [13]:
def k_means_score(X,k):

    scores = []

    for i in range(25):
        index_iter = k_means_cluster(X,k,i)
        score = SSE_SCORE(X, index_iter)

        scores.append(score)    
        print('index set number '+ str(i+1))  
        
    return scores

# First Attempt

### k = 1

In [21]:
k1_scores = k_means_score(X,1)

algorithm ran 1 times
index set number 1
algorithm ran 1 times
index set number 2
algorithm ran 1 times
index set number 3
algorithm ran 1 times
index set number 4
algorithm ran 1 times
index set number 5
algorithm ran 1 times
index set number 6
algorithm ran 1 times
index set number 7
algorithm ran 1 times
index set number 8
algorithm ran 1 times
index set number 9
algorithm ran 1 times
index set number 10
algorithm ran 1 times
index set number 11
algorithm ran 1 times
index set number 12
algorithm ran 1 times
index set number 13
algorithm ran 1 times
index set number 14
algorithm ran 1 times
index set number 15
algorithm ran 1 times
index set number 16
algorithm ran 1 times
index set number 17
algorithm ran 1 times
index set number 18
algorithm ran 1 times
index set number 19
algorithm ran 1 times
index set number 20
algorithm ran 1 times
index set number 21
algorithm ran 1 times
index set number 22
algorithm ran 1 times
index set number 23
algorithm ran 1 times
index set number 24
a

### k = 2

In [14]:
k2_scores = k_means_score(X,2)

algorithm ran 6 times
index set number 1
algorithm ran 18 times
index set number 2
algorithm ran 11 times
index set number 3
algorithm ran 15 times
index set number 4
algorithm ran 11 times
index set number 5
algorithm ran 11 times
index set number 6
algorithm ran 4 times
index set number 7
algorithm ran 8 times
index set number 8
algorithm ran 11 times
index set number 9
algorithm ran 10 times
index set number 10
algorithm ran 14 times
index set number 11
algorithm ran 8 times
index set number 12
algorithm ran 4 times
index set number 13
algorithm ran 9 times
index set number 14
algorithm ran 8 times
index set number 15
algorithm ran 9 times
index set number 16
algorithm ran 9 times
index set number 17
algorithm ran 18 times
index set number 18
algorithm ran 11 times
index set number 19
algorithm ran 42 times
index set number 20
algorithm ran 21 times
index set number 21
algorithm ran 5 times
index set number 22
algorithm ran 9 times
index set number 23
algorithm ran 29 times
index se

In [15]:
(np.mean(k2_scores), np.std(k2_scores))

(29523.884353138637, 2403.7209055247131)

### k = 3

In [16]:
k3_scores = k_means_score(X,3)

algorithm ran 7 times
index set number 1
algorithm ran 49 times
index set number 2
algorithm ran 4 times
index set number 3
algorithm ran 9 times
index set number 4
algorithm ran 3 times
index set number 5
algorithm ran 12 times
index set number 6
algorithm ran 23 times
index set number 7
algorithm ran 7 times
index set number 8
algorithm ran 6 times
index set number 9
algorithm ran 48 times
index set number 10
algorithm ran 6 times
index set number 11
algorithm ran 7 times
index set number 12
algorithm ran 9 times
index set number 13
algorithm ran 8 times
index set number 14
algorithm ran 6 times
index set number 15
algorithm ran 9 times
index set number 16
algorithm ran 9 times
index set number 17
algorithm ran 5 times
index set number 18
algorithm ran 47 times
index set number 19
algorithm ran 11 times
index set number 20
algorithm ran 10 times
index set number 21
algorithm ran 12 times
index set number 22
algorithm ran 8 times
index set number 23
algorithm ran 10 times
index set nu

In [17]:
(np.mean(k3_scores), np.std(k3_scores))

(24123.361322865236, 577.59008440051593)

### k = 4

In [18]:
k4_scores = k_means_score(X,4)

algorithm ran 5 times
index set number 1
algorithm ran 8 times
index set number 2
algorithm ran 4 times
index set number 3
algorithm ran 5 times
index set number 4
algorithm ran 12 times
index set number 5
algorithm ran 12 times
index set number 6
algorithm ran 8 times
index set number 7
algorithm ran 13 times
index set number 8
algorithm ran 24 times
index set number 9
algorithm ran 26 times
index set number 10
algorithm ran 6 times
index set number 11
algorithm ran 20 times
index set number 12
algorithm ran 9 times
index set number 13
algorithm ran 12 times
index set number 14
algorithm ran 17 times
index set number 15
algorithm ran 7 times
index set number 16
algorithm ran 3 times
index set number 17
algorithm ran 11 times
index set number 18
algorithm ran 10 times
index set number 19
algorithm ran 5 times
index set number 20
algorithm ran 13 times
index set number 21
algorithm ran 14 times
index set number 22
algorithm ran 10 times
index set number 23
algorithm ran 11 times
index s

In [19]:
(np.mean(k4_scores), np.std(k4_scores))

(20982.806400672336, 1365.5723109077023)

### k = 5

In [20]:
k5_scores = k_means_score(X,5)

algorithm ran 7 times
index set number 1
algorithm ran 29 times
index set number 2
algorithm ran 13 times
index set number 3
algorithm ran 2 times
index set number 4
algorithm ran 16 times
index set number 5
algorithm ran 4 times
index set number 6
algorithm ran 8 times
index set number 7
algorithm ran 5 times
index set number 8
algorithm ran 9 times
index set number 9
algorithm ran 19 times
index set number 10
algorithm ran 10 times
index set number 11
algorithm ran 27 times
index set number 12
algorithm ran 5 times
index set number 13
algorithm ran 13 times
index set number 14
algorithm ran 24 times
index set number 15
algorithm ran 5 times
index set number 16
algorithm ran 8 times
index set number 17
algorithm ran 11 times
index set number 18
algorithm ran 3 times
index set number 19
algorithm ran 14 times
index set number 20
algorithm ran 16 times
index set number 21
algorithm ran 13 times
index set number 22
algorithm ran 5 times
index set number 23
algorithm ran 9 times
index set

In [ ]:
(np.mean(k5_scores), np.std(k5_scores))

(18744.775451774636, 1766.6961023329713)

### k = 6

In [18]:
k6_scores = k_means_score(X,6)

algorithm ran 7 times
index set number 1
algorithm ran 1 times
index set number 2
algorithm ran 5 times
index set number 3
algorithm ran 14 times
index set number 4
algorithm ran 10 times
index set number 5
algorithm ran 15 times
index set number 6
algorithm ran 9 times
index set number 7
algorithm ran 14 times
index set number 8
algorithm ran 8 times
index set number 9
algorithm ran 7 times
index set number 10
algorithm ran 3 times
index set number 11
algorithm ran 12 times
index set number 12
algorithm ran 15 times
index set number 13
algorithm ran 6 times
index set number 14
algorithm ran 8 times
index set number 15
algorithm ran 19 times
index set number 16
algorithm ran 11 times
index set number 17
algorithm ran 9 times
index set number 18
algorithm ran 11 times
index set number 19
algorithm ran 14 times
index set number 20
algorithm ran 6 times
index set number 21
algorithm ran 4 times
index set number 22
algorithm ran 12 times
index set number 23
algorithm ran 17 times
index set

In [19]:
(np.mean(k6_scores), np.std(k6_scores))

(17145.57921889894, 2102.39530567477)

### k = 7

In [14]:
k7_scores = k_means_score(X,7)

algorithm ran 7 times
index set number 1
algorithm ran 5 times
index set number 2
algorithm ran 20 times
index set number 3
algorithm ran 3 times
index set number 4
algorithm ran 9 times
index set number 5
algorithm ran 26 times
index set number 6
algorithm ran 3 times
index set number 7
algorithm ran 12 times
index set number 8
algorithm ran 4 times
index set number 9
algorithm ran 8 times
index set number 10
algorithm ran 16 times
index set number 11
algorithm ran 14 times
index set number 12
algorithm ran 7 times
index set number 13
algorithm ran 8 times
index set number 14
algorithm ran 3 times
index set number 15
algorithm ran 6 times
index set number 16
algorithm ran 5 times
index set number 17
algorithm ran 14 times
index set number 18
algorithm ran 16 times
index set number 19
algorithm ran 13 times
index set number 20
algorithm ran 11 times
index set number 21
algorithm ran 24 times
index set number 22
algorithm ran 11 times
index set number 23
algorithm ran 18 times
index set

In [15]:
(np.mean(k7_scores), np.std(k7_scores))

(15500.005922611766, 1846.0415637756296)

### k = 8

In [16]:
k8_scores = k_means_score(X,8)

algorithm ran 7 times
index set number 1
algorithm ran 3 times
index set number 2
algorithm ran 6 times
index set number 3
algorithm ran 16 times
index set number 4
algorithm ran 8 times
index set number 5
algorithm ran 3 times
index set number 6
algorithm ran 16 times
index set number 7
algorithm ran 4 times
index set number 8
algorithm ran 3 times
index set number 9
algorithm ran 13 times
index set number 10
algorithm ran 12 times
index set number 11
algorithm ran 26 times
index set number 12
algorithm ran 14 times
index set number 13
algorithm ran 3 times
index set number 14
algorithm ran 7 times
index set number 15
algorithm ran 7 times
index set number 16
algorithm ran 18 times
index set number 17
algorithm ran 4 times
index set number 18
algorithm ran 14 times
index set number 19
algorithm ran 15 times
index set number 20
algorithm ran 5 times
index set number 21
algorithm ran 25 times
index set number 22
algorithm ran 19 times
index set number 23
algorithm ran 7 times
index set 

In [17]:
(np.mean(k8_scores), np.std(k8_scores))

(14379.661949730218, 1447.9948349638958)

### k = 9

In [17]:
k9_scores = k_means_score(X,9)

algorithm ran 7 times
index set number 1
algorithm ran 16 times
index set number 2
algorithm ran 19 times
index set number 3
algorithm ran 28 times
index set number 4
algorithm ran 11 times
index set number 5
algorithm ran 23 times
index set number 6
algorithm ran 26 times
index set number 7
algorithm ran 15 times
index set number 8
algorithm ran 19 times
index set number 9
algorithm ran 4 times
index set number 10
algorithm ran 11 times
index set number 11
algorithm ran 7 times
index set number 12
algorithm ran 18 times
index set number 13
algorithm ran 26 times
index set number 14
algorithm ran 7 times
index set number 15
algorithm ran 1 times
index set number 16
algorithm ran 16 times
index set number 17
algorithm ran 11 times
index set number 18
algorithm ran 20 times
index set number 19
algorithm ran 4 times
index set number 20
algorithm ran 28 times
index set number 21
algorithm ran 13 times
index set number 22
algorithm ran 3 times
index set number 23
algorithm ran 9 times
index

In [18]:
(np.mean(k9_scores), np.std(k9_scores))

(12807.717491626676, 1514.0988492359422)

### k = 10

In [19]:
k10_scores = k_means_score(X,10)

algorithm ran 7 times
index set number 1
algorithm ran 31 times
index set number 2
algorithm ran 31 times
index set number 3
algorithm ran 30 times
index set number 4
algorithm ran 3 times
index set number 5
algorithm ran 36 times
index set number 6
algorithm ran 10 times
index set number 7
algorithm ran 14 times
index set number 8
algorithm ran 33 times
index set number 9
algorithm ran 12 times
index set number 10
algorithm ran 8 times
index set number 11
algorithm ran 6 times
index set number 12
algorithm ran 10 times
index set number 13
algorithm ran 7 times
index set number 14
algorithm ran 34 times
index set number 15
algorithm ran 10 times
index set number 16
algorithm ran 4 times
index set number 17
algorithm ran 7 times
index set number 18
algorithm ran 24 times
index set number 19
algorithm ran 20 times
index set number 20
algorithm ran 3 times
index set number 21
algorithm ran 16 times
index set number 22
algorithm ran 10 times
index set number 23
algorithm ran 23 times
index

In [20]:
(np.mean(k10_scores), np.std(k10_scores))

(12212.127390800957, 1338.8366391639202)

### k = 11

In [21]:
k11_scores = k_means_score(X,11)

algorithm ran 7 times
index set number 1
algorithm ran 19 times
index set number 2
algorithm ran 10 times
index set number 3
algorithm ran 2 times
index set number 4
algorithm ran 4 times
index set number 5
algorithm ran 14 times
index set number 6
algorithm ran 10 times
index set number 7
algorithm ran 11 times
index set number 8
algorithm ran 14 times
index set number 9
algorithm ran 9 times
index set number 10
algorithm ran 7 times
index set number 11
algorithm ran 13 times
index set number 12
algorithm ran 16 times
index set number 13
algorithm ran 5 times
index set number 14
algorithm ran 15 times
index set number 15
algorithm ran 36 times
index set number 16
algorithm ran 10 times
index set number 17
algorithm ran 10 times
index set number 18
algorithm ran 2 times
index set number 19
algorithm ran 7 times
index set number 20
algorithm ran 21 times
index set number 21
algorithm ran 26 times
index set number 22
algorithm ran 19 times
index set number 23
algorithm ran 9 times
index 

In [22]:
(np.mean(k11_scores), np.std(k11_scores))

(11837.951284829069, 1194.2361242093366)

### k = 12

In [23]:
k12_scores = k_means_score(X,12)

algorithm ran 4 times
index set number 1
algorithm ran 9 times
index set number 2
algorithm ran 10 times
index set number 3
algorithm ran 12 times
index set number 4
algorithm ran 10 times
index set number 5
algorithm ran 10 times
index set number 6
algorithm ran 18 times
index set number 7
algorithm ran 8 times
index set number 8
algorithm ran 10 times
index set number 9
algorithm ran 11 times
index set number 10
algorithm ran 9 times
index set number 11
algorithm ran 6 times
index set number 12
algorithm ran 14 times
index set number 13
algorithm ran 18 times
index set number 14
algorithm ran 6 times
index set number 15
algorithm ran 25 times
index set number 16
algorithm ran 8 times
index set number 17
algorithm ran 9 times
index set number 18
algorithm ran 20 times
index set number 19
algorithm ran 16 times
index set number 20
algorithm ran 8 times
index set number 21
algorithm ran 4 times
index set number 22
algorithm ran 20 times
index set number 23
algorithm ran 5 times
index se

In [24]:
(np.mean(k12_scores), np.std(k12_scores))

(10995.354341982074, 1097.3352764297344)